In [137]:
from pandas.io.parsers.readers import TextFileReader
from sklearn import svm
from sklearn.model_selection import cross_val_score
import pandas as pd

############################
# タイタニックのCSVを使用しています
############################
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train_fe = df_train
df_test_fe = df_test

exclude_columns = [
    'Name',
    'Ticket',
    'PassengerId',
    'Cabin',
    'Survived'             
]
features = [c for c in df_train_fe if c not in exclude_columns]

svm = svm.SVC()
## Cross-Validation
df_train_fe[features]
df_train_fe["Survived"]

female_mean_age = df_train_fe[df_train_fe["Sex"] == "female"]["Age"].mean()
# df_train_fe[features].isnull().any()

# 欠損値補完
df_train_fe["Age"] = df_train_fe["Age"].fillna(df_train_fe["Age"].median())
import random
df_train_fe["Embarked"] = df_train_fe["Embarked"].fillna(random.choice(['S', 'C', 'Q']))

# pd.concat([df_train_fe, pd.get_dummies(df_train_fe["Embarked"], drop_first=True, prefix="Embarked")], axis=1)
df_train_fe = pd.concat([df_train_fe[features], pd.get_dummies(df_train_fe["Embarked"], prefix="Embarked")], axis=1).drop(columns=["Embarked"])
df_train_fe = pd.concat([df_train_fe, pd.get_dummies(df_train_fe["Sex"], drop_first=True)], axis=1).drop(columns=["Sex"])
df_train_fe

# 決定木で普通に分類した場合
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train_fe, df_train["Survived"], train_size=0.2, random_state=0)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
# model.predict(X_test)
model.score(X_test, y_test)

0.6984572230014026

In [139]:
# K分割してからSVMで予測した場合
score = cross_val_score(svm, df_train_fe, df_train["Survived"], scoring='roc_auc', cv=5, n_jobs=1)
score.mean()

0.7381530289165381